***TEXT TO MUSIC SAMPLE***

In [ ]:
import torch
torch.cuda.is_available()


True

In [ ]:
!pip install transformers accelerate gradio scipy soundfile

import torch
import numpy as np
import scipy.io.wavfile
import gradio as gr
from transformers import AutoProcessor, MusicgenForConditionalGeneration

In [ ]:
model_name = "facebook/musicgen-small"  # safest model

processor = AutoProcessor.from_pretrained(model_name)

device = "cpu"  # 🚨 FORCE CPU (NO CUDA)
print("Running on:", device)

model = MusicgenForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float32
).to(device)



# -------------------------
# PERFECT LOOP MAKER
# -------------------------
def make_loop(audio, sr, fade=1.0):
    fade_samples = int(sr * fade)

    fade_out = np.linspace(1, 0, fade_samples)
    fade_in = np.linspace(0, 1, fade_samples)

    audio[-fade_samples:] *= fade_out
    audio[:fade_samples] *= fade_in

    return audio


# -------------------------
# MAIN GENERATOR (FAST, 30 SEC)
# -------------------------
def generate_music(prompt, style, temperature, loop):
    duration = 45  # 30 seconds

    full_prompt = f"{style} style, {prompt}"

    # Faster, stable generation
    max_tokens = duration * 40   # 960 tokens for 30 sec

    inputs = processor(
    text=[full_prompt],
    padding=True,
    return_tensors="pt"
).to(device)
    with torch.inference_mode():
        audio_values = model.generate(
            **inputs,
            do_sample=True,
            temperature=temperature,
            max_new_tokens=max_tokens
        )

    audio = audio_values[0].cpu().numpy().flatten()
    sr = 32000

    if loop:
        audio = make_loop(audio, sr)

    file_path = "generated_track.wav"
    scipy.io.wavfile.write(file_path, sr, audio)

    return file_path, file_path


# -------------------------
# GRADIO UI
# -------------------------
styles = [
    "Ambient",
    "Horror",
    "Thriller",
    "Sci-Fi",
    "Dark Drone",
    "Cinematic",
    "Piano",
    "Lo-Fi"
]

with gr.Blocks() as demo:
    gr.Markdown("# 🎵 Ultra-Fast 45-Second AI Music Generator")
    gr.Markdown("Optimized for speed, looping & atmospheric background scoring.")

    prompt = gr.Textbox(
        label="Describe Your Music",
        placeholder="e.g. cold ambient drone with metallic echoes"
    )

    style = gr.Dropdown(styles, label="Music Style", value="Ambient")

    temp = gr.Slider(
        0.6, 1.5,
        value=1.0,
        step=0.1,
        label="Random Variation (Temperature)"
    )

    loop = gr.Checkbox(label="Make Seamless Loop", value=False)

    btn = gr.Button("Generate 45-Second Track 🎧")

    audio_out = gr.Audio(label="Preview")
    download_out = gr.File(label="Download Track")

    btn.click(
        generate_music,
        inputs=[prompt, style, temp, loop],
        outputs=[audio_out, download_out]
    )

demo.launch()

Running on: cpu
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://19ee1dc669f1e71868.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
